In [1]:
!pip install ultralytics==8.3.23


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 15.0 MB/s eta 0:00:00


In [34]:
import zipfile
with zipfile.ZipFile("train_dataset (2).zip", 'r') as zip_ref:
  zip_ref.extractall('train_dataset (2)')

In [4]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [36]:
IMAGES_DIR = '/content/train_dataset (2)/train_dataset/cv_open_dataset/open_img'  # Путь к вашему датасету с изображениями
MASKS_DIR = '/content/train_dataset (2)/train_dataset/cv_open_dataset/open_msk'  # Путь к вашему датасету с масками
OUTPUT_DIR = './datasets/train_data(2)'  # Путь к выходной директории
TRAIN_SIZE = 0.85  # Процент обучающей выборки
TEST_SIZE = 0.2

In [37]:
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

In [38]:
image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(MASKS_DIR) if f.endswith('.png')]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

In [39]:
train_images, val_images = train_test_split(image_files, train_size=TRAIN_SIZE, random_state=42)

In [40]:
def convert_mask_to_yolo(mask_path, out):
    # Открываем изображение
    image = cv2.imread(mask_path)

    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return

    height, width = image.shape[:2]

    # Создаем маску для черного цвета
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))

    # Создаем новое изображение, где черный цвет остается, а остальные цвета становятся белыми
    new_image = np.ones_like(image) * 255  # Начинаем с белого изображения
    new_image[black_mask > 0] = [0, 0, 0]  # Заменяем черные пиксели

    # Преобразуем в градации серого для нахождения контуров
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Находим контуры
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Сохраняем контуры в текстовом формате
    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, out, output_file_name)

    with open(output_file_path, 'w') as f:
        for index, contour in enumerate(contours):
            # Получаем координаты всех точек контура
            contour_points = contour.reshape(-1, 2)
            # Нормализуем координаты
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

In [41]:
# Копирование изображений и масок в соответствующие папки
for img in train_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/train', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/train')

In [42]:
for img in val_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/val', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/val')

In [43]:
data_yaml_content = f"""
train: train_data/images/train
val: train_data/images/val

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Датасет успешно разбит и сохранен в структуре проекта.")

Датасет успешно разбит и сохранен в структуре проекта.


In [27]:


# Initialize the YOLO model
model = YOLO("yolo11n.pt")

# Tune hyperparameters on COCO8 for 30 epochs
model.tune(data="coco8.yaml", epochs=10, iterations=10, optimizer="AdamW", plots=False, save=False, val=False)


Tuner: Initialized Tuner instance with 'tune_dir=runs/detect/tune4'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
Saved runs/detect/tune4/tune_scatter_plots.png
Saved runs/detect/tune4/tune_fitness.png

Tuner: 1/10 iterations complete ✅ (66.54s)
Tuner: Results saved to runs/detect/tune4
Tuner: Best fitness=0.0 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.0, 'metrics/recall(B)': 0.0, 'metrics/mAP50(B)': 0.0, 'metrics/mAP50-95(B)': 0.0, 'val/box_loss': 3.91623, 'val/cls_loss': 18.55877, 'val/dfl

In [28]:
model = YOLO("yolo11n-seg.pt")

In [58]:
import os
import random
from torchvision import transforms
from PIL import Image

# Пути к папкам с изображениями и масками
images_dir = '/content/train_dataset (2)/train_dataset/cv_open_dataset/open_img'
masks_dir = '/content/train_dataset (2)/train_dataset/cv_open_dataset/open_msk'

# Директории для хранения аугментированных данных
augmented_images_dir = '/content/augmented(2)/images'
augmented_masks_dir = '/content/augmented(2)/masks'

# Создаем директории для аугментированных данных
os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_masks_dir, exist_ok=True)

# Список всех изображений и масок
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(masks_dir) if f.endswith(('.jpg', '.png'))]

# Сортируем файлы по имени, чтобы изображения и маски соответствовали друг другу
image_files.sort()
mask_files.sort()

# Проверка на соответствие
assert len(image_files) == len(mask_files), "Количество изображений не совпадает с количеством масок."

# Количество изображений для аугментации (например, 20% от всех данных)
num_augmented = int(1* len(image_files))

# Список изображений для аугментации
augmented_image_files = random.sample(image_files, num_augmented)

# Трансформации для аугментации (учитываем, что маска должна оставаться розово-чёрной)
image_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.RandomResizedCrop(size=640, scale=(0.8, 1.0)),
    transforms.RandomApply([
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0)
    ], p=0.3)
])

mask_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(45),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.RandomResizedCrop(size=640, scale=(0.8, 1.0)),
])
def augment_image_and_mask(image_path, mask_path):
    # Загружаем изображение и маску
    image = Image.open(image_path).convert('RGB')  # Оставляем изображение цветным
    mask = Image.open(mask_path)  # Маска тоже остаётся в формате RGB

    # Применяем трансформации
    augmented_image = image_transform(image)
    augmented_mask = mask_transform(mask)

    return augmented_image, augmented_mask

# Создание аугментированных данных
for image_file in augmented_image_files:
    # Путь к изображению
    image_path = os.path.join(images_dir, image_file)

    # Путь к маске: заменяем расширение, если необходимо
    mask_path = os.path.join(masks_dir, image_file.replace('.jpg', '.png'))

    # Проверяем существование файлов
    if not os.path.exists(image_path):
        print(f"Изображение не найдено: {image_path}")
        continue
    if not os.path.exists(mask_path):
        print(f"Маска не найдена: {mask_path}")
        continue

    # Получаем аугментированные изображения и маски
    augmented_image, augmented_mask = augment_image_and_mask(image_path, mask_path)

    # Сохранение аугментированных изображений и масок
    augmented_image_path = os.path.join(augmented_images_dir, image_file)
    augmented_mask_path = os.path.join(augmented_masks_dir, image_file.replace('.jpg', '.png'))  # Сохраняем как .png

    # Сохраняем результат
    augmented_image.save(augmented_image_path)
    augmented_mask.save(augmented_mask_path)

    print(f"Аугментированное изображение и маска сохранены: {image_file}")

print(f"Создано {num_augmented} аугментированных изображений и масок.")


Аугментированное изображение и маска сохранены: F4_2_3_1.ts_f_500.jpg
Аугментированное изображение и маска сохранены: 93_1710079053_0.png
Аугментированное изображение и маска сохранены: F7_1_1_1.ts_f_500.jpg
Аугментированное изображение и маска сохранены: 20_1709344621_0.png
Аугментированное изображение и маска сохранены: F7_2_1_1.ts_f_500.jpg
Аугментированное изображение и маска сохранены: 103_1711186053_0.png
Аугментированное изображение и маска сохранены: 296_dirty1.png
Аугментированное изображение и маска сохранены: 1711090673_0.jpg
Аугментированное изображение и маска сохранены: 87_1709344621_0.png


AttributeError: 'Image' object has no attribute 'shape'

In [59]:
import os
import random
from torchvision import transforms
from PIL import Image

images_dir = '/content/train_dataset (2)/train_dataset/cv_open_dataset/open_img'
masks_dir = '/content/train_dataset (2)/train_dataset/cv_open_dataset/open_msk'

# Директории для хранения аугментированных данных
augmented_images_dir = '/content/augmented(2)/images'
augmented_masks_dir = '/content/augmented(2)/masks'
# Создаем директории для аугментированных данных
os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_masks_dir, exist_ok=True)

# Список всех изображений и масок
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.txt'))]
mask_files = [f for f in os.listdir(masks_dir) if f.endswith(('.jpg', '.png', '.txt'))]

# Сортируем файлы по имени, чтобы изображения и маски соответствовали друг другу
image_files.sort()
mask_files.sort()

# Проверка на соответствие
assert len(image_files) == len(mask_files), "Количество изображений не совпадает с количеством масок."

# Количество изображений для аугментации (например, 20% от всех данных)
num_augmented = int(1* len(image_files))

# Список изображений для аугментации
augmented_image_files = random.sample(image_files, num_augmented)


# Дополнительные трансформации для аугментации
image_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.RandomResizedCrop(size=640, scale=(0.8, 1.0)),
    transforms.RandomApply([
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0)
    ], p=0.3)
])

mask_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(45),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.RandomResizedCrop(size=640, scale=(0.8, 1.0)),
])



# Функция для применения аугментаций
def augment_image_and_mask(image_path, mask_path):
    # Загружаем изображение и маску
    image = Image.open(image_path).convert('RGB')  # Оставляем изображение цветным
    mask = Image.open(mask_path).convert('RGB')   # Маска тоже остаётся в формате RGB

    # Применяем трансформации
    augmented_image = image_transform(image)
    augmented_mask = mask_transform(mask)

    return augmented_image, augmented_mask

# Создание аугментированных данных
for image_file in augmented_image_files:
    # Путь к изображению
    image_path = os.path.join(images_dir, image_file)

    # Путь к маске: заменяем расширение, если необходимо
    mask_path = os.path.join(masks_dir, image_file.replace('.jpg', '.png'))

    # Проверяем существование файлов
    if not os.path.exists(image_path):
        print(f"Изображение не найдено: {image_path}")
        continue
    if not os.path.exists(mask_path):
        print(f"Маска не найдена: {mask_path}")
        continue

    # Получаем аугментированные изображения и маски
    augmented_image, augmented_mask = augment_image_and_mask(image_path, mask_path)

    # Сохранение аугментированных изображений и масок
    augmented_image_path = os.path.join(augmented_images_dir, image_file)
    augmented_mask_path = os.path.join(augmented_masks_dir, image_file.replace('.jpg', '.png'))  # Сохраняем как .png

    # Сохраняем результат
    augmented_image.save(augmented_image_path)
    augmented_mask.save(augmented_mask_path)

    print(f"Аугментированное изображение и маска сохранены: {image_file}")

print(f"Создано {num_augmented} аугментированных изображений и масок.")

Аугментированное изображение и маска сохранены: 1712741269_0.jpg
Аугментированное изображение и маска сохранены: 81_1709262422_0.png
Аугментированное изображение и маска сохранены: 414_1709262422_0.png
Аугментированное изображение и маска сохранены: 25_1709888254_0.png


AttributeError: 'Image' object has no attribute 'shape'

In [60]:
import os
import random
from torchvision import transforms
from PIL import Image

# Пути к папкам с изображениями и масками
images_dir = '/content/train_dataset (2)/train_dataset/cv_open_dataset/open_img'
masks_dir = '/content/train_dataset (2)/train_dataset/cv_open_dataset/open_msk'

# Директории для хранения аугментированных данных
augmented_images_dir = '/content/augmented(2)/images'
augmented_masks_dir = '/content/augmented(2)/masks'

# Создаем директории для аугментированных данных
os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_masks_dir, exist_ok=True)

# Список всех изображений и масок
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(masks_dir) if f.endswith(('.jpg', '.png'))]

# Сортируем файлы по имени, чтобы изображения и маски соответствовали друг другу
image_files.sort()
mask_files.sort()

# Проверка на соответствие
assert len(image_files) == len(mask_files), "Количество изображений не совпадает с количеством масок."

# Количество изображений для аугментации (например, 20% от всех данных)
num_augmented = int(1 * len(image_files))

# Список изображений для аугментации
augmented_image_files = random.sample(image_files, num_augmented)

# Дополнительные трансформации для аугментации
image_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.RandomResizedCrop(size=640, scale=(0.8, 1.0)),
])

random_erasing = transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0)

mask_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(45),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.RandomResizedCrop(size=640, scale=(0.8, 1.0)),
])

# Функция для применения аугментаций
def augment_image_and_mask(image_path, mask_path):
    # Загружаем изображение и маску
    image = Image.open(image_path).convert('RGB')  # Оставляем изображение цветным
    mask = Image.open(mask_path).convert('RGB')   # Маска остаётся в формате RGB

    # Применяем трансформации к изображению
    augmented_image = image_transform(image)
    augmented_image = transforms.ToTensor()(augmented_image)  # Преобразуем в тензор для RandomErasing
    augmented_image = random_erasing(augmented_image)  # Применяем RandomErasing
    augmented_image = transforms.ToPILImage()(augmented_image)  # Обратно в формат PIL.Image

    # Применяем трансформации к маске
    augmented_mask = mask_transform(mask)

    return augmented_image, augmented_mask

# Создание аугментированных данных
for image_file in augmented_image_files:
    # Путь к изображению
    image_path = os.path.join(images_dir, image_file)

    # Путь к маске: заменяем расширение, если необходимо
    mask_path = os.path.join(masks_dir, image_file.replace('.jpg', '.png'))

    # Проверяем существование файлов
    if not os.path.exists(image_path):
        print(f"Изображение не найдено: {image_path}")
        continue
    if not os.path.exists(mask_path):
        print(f"Маска не найдена: {mask_path}")
        continue

    # Получаем аугментированные изображения и маски
    augmented_image, augmented_mask = augment_image_and_mask(image_path, mask_path)

    # Сохранение аугментированных изображений и масок
    augmented_image_path = os.path.join(augmented_images_dir, image_file)
    augmented_mask_path = os.path.join(augmented_masks_dir, image_file.replace('.jpg', '.png'))  # Сохраняем как .png

    # Сохраняем результат
    augmented_image.save(augmented_image_path)
    augmented_mask.save(augmented_mask_path)

    print(f"Аугментированное изображение и маска сохранены: {image_file}")

print(f"Создано {num_augmented} аугментированных изображений и масок.")


Аугментированное изображение и маска сохранены: 259_1709186522_0.png
Аугментированное изображение и маска сохранены: F1_1_5_2.ts_f_1000.jpg
Аугментированное изображение и маска сохранены: F5_1_3_1.ts_f_500.jpg
Аугментированное изображение и маска сохранены: 1715158095_0.jpg
Аугментированное изображение и маска сохранены: F5_2_3_1.ts_f_500.jpg
Аугментированное изображение и маска сохранены: F4_2_2_2.ts_f_500.jpg
Аугментированное изображение и маска сохранены: 265_1709255042_0.png
Аугментированное изображение и маска сохранены: 20_1709344621_0.png
Аугментированное изображение и маска сохранены: 419_1709339881_0.png
Аугментированное изображение и маска сохранены: 33_1710813452_0.png
Аугментированное изображение и маска сохранены: 34_1710835053_0.png
Аугментированное изображение и маска сохранены: F2_2_3_2.ts_f_1000.jpg
Аугментированное изображение и маска сохранены: F2_2_3_1.ts_f_1000.jpg
Аугментированное изображение и маска сохранены: F4_2_3_2.ts_f_1000.jpg
Аугментированное изображение и

In [61]:
# Загрузка данных для тренировки
data_yaml_content = f"""
train: train_data/images/train
val: train_data/images/val

nc: 1  # Количество классов (1 для загрязнения)
names: ['contaminated']  # Название класса
"""

# Сохранение файла с настройками для данных
with open('data.yaml', 'w') as f:
    f.write(data_yaml_content)

In [62]:
train_results = model.train(
    data="./data.yaml",
    epochs=20,
    imgsz=512,
    device="cpu",
    batch=8,
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_momentum=0.8,
    warmup_epochs=3,
    cls=0.5,
    box=7.5,
    dfl=1.5,
    hsv_h=0.015,
    hsv_v=0.4,
    hsv_s=0.7,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    mosaic=0.5,
    fliplr=0.5,
    patience=20,
    cos_lr=True,
    mixup=0.0,  # Пробуем смешивание данных
)


New https://pypi.org/project/ultralytics/8.3.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=./data.yaml, epochs=20, time=None, patience=20, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=cpu, workers=0, project=None, name=train8333, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

train: Scanning /content/datasets/train_data/labels/train.cache... 136 images, 103 backgrounds, 0 corrupt: 100%|██████████| 136/136 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/datasets/train_data/labels/val.cache... 34 images, 25 backgrounds, 0 corrupt: 100%|██████████| 34/34 [00:00<?, ?it/s]

Plotting labels to runs/segment/train8333/labels.jpg... 


...

      20/20         0G      1.128       1.64      2.994      1.387          2        512: 100%|██████████| 17/17 [01:36<00:00,  5.70s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:07<00:00,  2.42s/it]

                   all         34         17      0.429      0.353      0.405      0.255      0.859      0.235      0.311      0.228



20 epochs completed in 0.590 hours.
Optimizer stripped from runs/segment/train8333/weights/last.pt, 6.0MB
Optimizer stripped from runs/segment/train8333/weights/best.pt, 6.0MB

Validating runs/segment/train8333/weights/best.pt...
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:08<00:00,  2.83s/it]


                   all         34         17      0.403      0.358      0.404      0.257       0.87      0.235      0.319       0.23
Speed: 1.3ms preprocess, 196.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/segment/train8333


In [63]:
results = model.val(data="./data.yaml")
print(results)

Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


val: Scanning /content/datasets/train_data/labels/val.cache... 34 images, 25 backgrounds, 0 corrupt: 100%|██████████| 34/34 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:07<00:00,  1.58s/it]


                   all         34         17      0.551      0.294      0.351      0.288      0.551      0.294      0.325      0.223
Speed: 2.5ms preprocess, 167.6ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/segment/train83332
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b20112122f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.0220

In [64]:
def infer_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)

    # Инференс
    return model(image)

In [65]:
def create_mask(image_path, results):
    # Загружаем изображение и переводим в градации серого
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Создаем пустую маску с черным фоном
    mask = np.zeros((height, width), dtype=np.uint8)

    # Проходим по результатам и создаем маску
    for result in results:
        masks = result.masks  # Получаем маски из результатов
        if masks is not None:
            for mask_array in masks.data:  # Получаем маски как массивы
                mask_i = mask_array.numpy()  # Преобразуем маску в numpy массив

                # Изменяем размер маски под размер оригинального изображения
                mask_i_resized = cv2.resize(mask_i, (width, height), interpolation=cv2.INTER_LINEAR)

                # Накладываем маску на пустую маску (255 для белого)
                mask[mask_i_resized > 0] = 255

    return mask

In [66]:
model.save('model_nornikel.pt')

print("Модель успешно сохранена в формате .pt")


Модель успешно сохранена в формате .pt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')